In [2]:
import pandas as pd
import numpy as np
import time
import datetime as dt
import os
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from dtw import dtw
from sklearn.metrics import pairwise_distances
PRE_NAME = "onemin_ohlc_"
BEGIN_TIME = "09:00:00"
END_TIME = "11:00:00"
def load_data():
    X = []
    Y = []
    # df = pd.read_csv(os.path.join('dataset', '2327', PRE_NAME+"20180612.csv"))
    # mask = (df.loc[:, "time"] >= BEGIN_TIME) & (df.loc[:, "time"] <= END_TIME)
    # front_df = df[mask].loc[:, "return"]
    # end_df = df[~mask].loc[:, "return"]
    # X.append(np.array(front_df))
    # Y.append(np.array(end_df))
    # """我是分隔線^^~"""
    # df = pd.read_csv(os.path.join('dataset', '2327', PRE_NAME+"20180613.csv"))
    # mask = (df.loc[:, "time"] >= BEGIN_TIME) & (df.loc[:, "time"] <= END_TIME)
    # front_df = df[mask].loc[:, "return"]
    # end_df = df[~mask].loc[:, "return"]
    # X.append(np.array(front_df))
    # Y.append(np.array(end_df))
    
    
    sid = '2327'
    for file in os.listdir(os.path.join('dataset', sid)):
        # print(file)
        df = pd.read_csv(os.path.join('dataset', sid, file))
        mask = (df.loc[:, "time"] >= BEGIN_TIME) & (df.loc[:, "time"] <= END_TIME)
        front_df = df[mask].loc[:, "return"]
        # print(front_df)
        # exit()
        end_df = df[~mask].loc[:, "return"]
        if len(front_df) == 121:
        #     print(df.to_string())
        #     exit()
        # print(np.array(front_df).shape)
            X.append(np.array(front_df))
            Y.append(np.array(end_df))
    # print(end_df)
    # df = pd.read_csv(os.path.join('dataset', '2327', PRE_NAME+"20180612.csv"))
    # print(len(X))
    # print(len(X[0]))
    # X = np.array(X)
    # print(X.shape)
        
    # exit()
    return np.array(X), np.array(Y)

def dtw_d(X, Y):
    manhattan_distance = lambda x, y: np.abs(x - y)
    d, cost_matrix, acc_cost_matrix, path = dtw(X, Y, dist=manhattan_distance)
    return d

def dtw_affinity(X):
    return pairwise_distances(X, metric=dtw_d)

X, Y = load_data()

ac = AgglomerativeClustering(n_clusters = 10,
                             affinity = dtw_affinity,
                             linkage = 'complete')
X_label = ac.fit_predict(X)


In [3]:
X_label

array([0, 0, 0, 4, 9, 3, 2, 3, 9, 7, 5, 0, 9, 0, 9, 9, 7, 1, 9, 9, 1, 0,
       7, 1, 0, 1, 9, 9, 1, 7, 4, 4, 9, 4, 2, 9, 1, 1, 7, 9, 2, 7, 3, 1,
       0, 4, 1, 1, 6, 9, 4, 9, 9, 4, 9, 1, 9, 1, 7, 7, 3, 1, 1, 0, 4, 1,
       6, 3, 1, 3, 3, 1, 0, 4, 7, 7, 6, 3, 7, 0, 2, 2, 8, 7, 0, 1, 9, 7,
       0, 1, 9, 1, 1, 1, 1, 1, 1, 9, 3, 9, 1, 4, 0, 1, 0, 1, 3, 4, 4, 1,
       7, 7, 9, 3, 1, 1, 4, 3, 2, 1, 9, 1, 1, 2, 2, 7, 4, 3, 1, 1, 0, 7,
       7, 1, 7, 3, 1, 4, 1, 1, 9, 1, 1, 9, 7, 7, 7, 4, 1, 4, 7, 4, 7, 7,
       4, 7, 1, 1, 4, 1, 1, 1, 9, 4, 7, 9, 3, 1, 3, 1, 9], dtype=int64)